In [1]:
import numpy as np
import pandas as pd
import os
import json
import ast
import random
import tensorflow as tf
import tensorflow_addons as tfa
from bpmll import bp_mll_loss
import sklearn_json as skljson
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sys
sys.path.append('../ThresholdFunctionLearning')    ## Append path to the ThresholdFunctionLearning directory to the interpreters
                                                   ## search path
from threshold_learning import predict_test_labels_binary    ## Import the 'predict_test_labels_binary()' function from the 
from threshold_learning import predict_labels_binary         ## threshold_learning library

## Load the Cross Entropy LSTM

In [2]:
## Define the LSTM RNN architecture
num_labels = 13
max_length = 100
num_unique_words = 2711

model_LSTM = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(num_unique_words, 32, input_length = max_length),
    tf.keras.layers.LSTM(16, return_sequences = False, return_state = False),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_labels, activation = 'sigmoid')
])

optim = tf.keras.optimizers.Adam(lr=0.0001)
#optim_func_LSTM = tf.keras.optimizers.Adagrad(
#    learning_rate = 0.001, initial_accumulator_value = 0.1, epsilon = 1e-07,
#    name = 'Adagrad')

#metrics = tfa.metrics.hamming_loss_fn(mode = 'multi-label')
model_LSTM.compile(loss = 'categorical_crossentropy', optimizer = optim)

In [3]:
# Restore the weights
model_LSTM.load_weights('Models/Cross_Entropy/LSTM_weights')

In [4]:
## Load the pre-processed data
file_object = open('RNN_data_dict.json',)
RNN_data_dict = json.load(file_object)
RNN_data_dict = ast.literal_eval(RNN_data_dict)
train_padded = np.array(RNN_data_dict['train_padded'])
test_padded = np.array(RNN_data_dict['test_padded'])
Y_train = np.array(RNN_data_dict['Y_train'])
Y_test = np.array(RNN_data_dict['Y_test'])

In [5]:
## Using a constant 0.5 threshold function, get the hamming loss for the trained network on the test set
predictions = model_LSTM.predict(test_padded)
predictions_binary = model_LSTM.predict(test_padded)
for i in range(Y_test.shape[0]):
    for j in range(Y_test.shape[1]):
        if predictions_binary[i, j] > 0.5:
            predictions_binary[i, j] = 1
        else:
            predictions_binary[i, j] = 0

# Get the hamming loss
metrics.hamming_loss(Y_test, predictions_binary)

0.5024038461538461

In [6]:
## Load the learned threshold function and get test-set hamming loss
Y_test_pred = model_LSTM.predict(test_padded)
threshold_filepath = 'Models/Cross_Entropy/threshold_LSTM.json'
threshold_function = skljson.from_json(threshold_filepath)
test_labels_binary = predict_labels_binary(Y_test_pred, threshold_function)

metrics.hamming_loss(Y_test, test_labels_binary)

C:\Users\rober\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass fit_intercept={'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False} as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


0.17548076923076922

## Load the BP-MLL Bidirectional LSTM (trained on many epochs)

In [8]:
## Define the bidirectional LSTM RNN architecture
model_biLSTM_bpmll = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(num_unique_words, 32, input_length = max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences = False, return_state = False)),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_labels, activation = 'sigmoid')
])

optim_bpmll = tf.keras.optimizers.Adam(lr=0.001)
#optim_bpmll = tf.keras.optimizers.Adagrad(
#    learning_rate = 0.1, initial_accumulator_value = 0.1, epsilon = 1e-07,
#    name = 'Adagrad')

#optim = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum = 0.8, epsilon=1e-07,)

metric = tfa.metrics.HammingLoss(mode = 'multilabel', threshold = 0.5)
model_biLSTM_bpmll.compile(loss = bp_mll_loss, optimizer = optim_bpmll, metrics = metric)

In [12]:
model_biLSTM_bpmll.load_weights('Models/BPMLL/biLSTM_bpmll_weights_manyEpochs')

In [14]:
## Load the pre-processed data
file_object_reduced = open('RNN_data_dict_reduced.json',)
RNN_data_dict_reduced = json.load(file_object_reduced)
RNN_data_dict_reduced = ast.literal_eval(RNN_data_dict_reduced)
train_padded_hasLabel = np.array(RNN_data_dict_reduced['train_padded_hasLabel'])
test_padded_hasLabel = np.array(RNN_data_dict_reduced['test_padded_hasLabel'])
Y_train_hasLabel = np.array(RNN_data_dict_reduced['Y_train_hasLabel'])
Y_test_hasLabel = np.array(RNN_data_dict_reduced['Y_test_hasLabel'])

In [15]:
## Using a constant 0.5 threshold function, get the hamming loss for the trained network on the test set
predictions = model_biLSTM_bpmll.predict(test_padded_hasLabel)
predictions_binary = model_biLSTM_bpmll.predict(test_padded_hasLabel)
for i in range(Y_test_hasLabel.shape[0]):
    for j in range(Y_test_hasLabel.shape[1]):
        if predictions_binary[i, j] > 0.5:
            predictions_binary[i, j] = 1
        else:
            predictions_binary[i, j] = 0

# Get the hamming loss
metrics.hamming_loss(Y_test_hasLabel, predictions_binary)

0.15034965034965034